# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846833869919                   -0.70    4.8
  2   -7.852324127405       -2.26       -1.53    1.0
  3   -7.852646074420       -3.49       -2.52    3.2
  4   -7.852646677858       -6.22       -3.37    2.0
  5   -7.852646685874       -8.10       -4.77    1.2
  6   -7.852646686723       -9.07       -5.20    3.8
  7   -7.852646686730      -11.19       -6.10    1.5
  8   -7.852646686730      -12.54       -7.29    2.5
  9   -7.852646686730      -14.75       -7.50    2.0
 10   -7.852646686730   +  -14.75       -8.64    2.0
 11   -7.852646686730   +    -Inf       -9.55    2.2
 12   -7.852646686730      -14.75      -10.19    2.2
 13   -7.852646686730   +  -14.45      -10.54    1.2
 14   -7.852646686730      -14.75      -11.51    1.8
 15   -7.852646686730   +    -Inf      -12.03    2.8


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846865228085                   -0.70           4.8
  2   -7.852528953265       -2.25       -1.63   0.80    2.0
  3   -7.852637296646       -3.97       -2.71   0.80    1.0
  4   -7.852646514579       -5.04       -3.30   0.80    2.2
  5   -7.852646674393       -6.80       -4.13   0.80    1.0
  6   -7.852646686455       -7.92       -4.76   0.80    1.8
  7   -7.852646686725       -9.57       -5.50   0.80    1.8
  8   -7.852646686730      -11.35       -6.54   0.80    1.5
  9   -7.852646686730      -12.64       -7.64   0.80    2.5
 10   -7.852646686730      -14.45       -8.16   0.80    2.2
 11   -7.852646686730   +    -Inf       -8.85   0.80    1.2
 12   -7.852646686730      -14.75       -9.27   0.80    1.8
 13   -7.852646686730   +  -14.45      -10.61   0.80    1.5
 14   -7.852646686730   +    -Inf      -11.04   0.80    2.8
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.359635e+01     3.130224e+00
 * time: 0.5113751888275146
     1     9.989111e-01     1.691151e+00
 * time: 0.7747170925140381
     2    -1.572958e+00     1.912545e+00
 * time: 0.8104970455169678
     3    -3.951798e+00     1.661732e+00
 * time: 0.8636701107025146
     4    -5.206350e+00     1.489098e+00
 * time: 0.9163761138916016
     5    -6.949500e+00     1.060034e+00
 * time: 0.9747982025146484
     6    -7.491058e+00     6.398959e-01
 * time: 1.0274531841278076
     7    -7.706429e+00     4.616394e-01
 * time: 1.0712461471557617
     8    -7.785557e+00     1.467471e-01
 * time: 1.107795000076294
     9    -7.821048e+00     1.020928e-01
 * time: 1.1443190574645996
    10    -7.838458e+00     7.520775e-02
 * time: 1.180110216140747
    11    -7.844678e+00     4.721835e-02
 * time: 1.216008186340332
    12    -7.849372e+00     2.578113e-02
 * time: 1.2520902156829834
    13    -7.851380e+00     1.952536e-02
 * time: 1.2884831428527

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846806246997                   -0.70    4.5
  2   -7.852315156191       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686683                   -2.55
  2   -7.852646686730      -10.33       -5.88
  3   -7.852646686730   +  -14.75      -12.50


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.958923498771137e-13
|ρ_newton - ρ_scfv| = 2.8546424869035893e-13
|ρ_newton - ρ_dm|   = 2.185779698181387e-9
